<h1 style="color:darkblue;background-color:white;font-family:Pacifico;font-size:400%;text-align:center">Disneyland</h1>

* <h1 style="background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:left">Disneyland Park is most certainly a land of enchantment where children and the young at heart find that dreams really do come true. Disneyland Park is a seamless blend of yesterday, today and tomorrow. Smiling train conductors, marching bands and the clip-clop of horse drawn carriages bring you back to carefree days. You can also rocket through the galaxy, board a runaway train or trek through the jungle. Laughter is always in the air, with friendly smiles all around. The secret to Disneyland is its ability to change yet, remain the same. As Walt Disney said "Disneyland will never be completed. It will continue to grow as long as there is imagination left in the world."</h1>

  ![mickey-character-castle-5x2-1.jpg](https://i.postimg.cc/ZR2GpjZP/mickey-character-castle-5x2-1.jpg)

* <h1 style="background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:left">we will try through this Dataset to figure out some informative insights about Disneyland, extract some conclusions from Disneyland reviews and make the sentiment analysis"</h1>


<h1 style="color:white;background-color:black;font-family:Impact;font-size:350%;text-align:center">Table Of Content</h1>



* [1. Import & Explore](#1)

* [2. Feature Engineering](#2)

* [3. Data Anlaysis & Visualizations](#3)
    
* [4. Sentiment in Reviews](#4)
    
* [5. Words in Negative Reviews](#5)



<a id="1"></a>
<h1 style="color:green;background-color:white;font-family:Pacifico;font-size:400%;text-align:center">Import & Explore</h1>


In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
import re
from PIL import Image

In [ ]:
url='../input/disneyland-reviews/DisneylandReviews.csv'
df=pd.read_csv(url,encoding="cp1252")
df

In [ ]:
df=df.loc[df['Year_Month']=='missing']
df

* <h1 style="background-color:white;font-family:Gill Sans MT;font-size:200%;text-align:left">Removing Nulls & Duplicates</h1>


In [ ]:
url='../input/disneyland-reviews/DisneylandReviews.csv'
df=pd.read_csv(url,encoding="cp1252",na_values=['missing'])
#== 
df=df.dropna().reset_index()
print ("\nMissing values :  ", df.isnull().sum().values.sum())


In [ ]:
df.drop_duplicates(subset='Review_Text', inplace=True, keep='first')

* <h1 style="background-color:white;font-family:Gill Sans MT;font-size:200%;text-align:left">Data Summary</h1>

In [ ]:
print ("Rows     : " ,df.shape[0])
print ("Columns  : " ,df.shape[1])
print ("\nFeatures : \n" ,df.columns.tolist())
print ("\nMissing values :  ", df.isnull().sum().values.sum())
print ("\nUnique values :  \n",df.nunique())

In [ ]:
df['Branch'].value_counts()

* There is some bias in the data to Branches feature value counts aren't fairly distributed

<a id="2"></a>

<h1 style="color:dodgerblue;background-color:white;font-family:Pacifico;font-size:400%;text-align:center">Feature 
Engineering</h1>

<h1 style="color:dodgerblue;background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:center">ـــــــــــ</h1>

* <h1 style="background-color:white;font-family:Gill Sans MT;font-size:200%;text-align:left">Get time-series features</h1>

In [ ]:
# new data frame with split value columns 
new = df["Year_Month"].str.split("-", n = 1, expand = True) 
  
# making separate year column from new data frame 
df["year"]= new[0] 
  
# making separate mounth column from new data frame 
df["mounth"]= new[1] 
  
# Dropping old feature  
df.drop(columns =["Year_Month"], inplace = True) 

In [ ]:
df['mounth']=df['mounth'].astype('int64')
df['quarter']=df['mounth'].apply(lambda x:1 if x<=3 else (2 if 3<x<=6 else (3 if 6<x<=9 else (4 if 9<x<=12 else x))))

* <h1 style="background-color:white;font-family:Gill Sans MT;font-size:200%;text-align:left">Get Review Length Feature</h1>

In [ ]:
df['Review_Text'].values[2:3]

In [ ]:
df['review length']=df['Review_Text'].apply(lambda x:len(x))


* <h1 style="background-color:white;font-family:Gill Sans MT;font-size:200%;text-align:left">Get Sentiment Analysis Feature</h1>

In [ ]:
#==
features = df['Review_Text'].values
#=== 
processed_features = []

for sentence in range(0, len(features)):
    # Remove all the Http: urls
    processed_feature = re.sub('(https?://\S+)', '', str(features[sentence]))
    
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', processed_feature)

    # Remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)
#=== 
df3=pd.DataFrame()
df3['reviews']=processed_features
#=======
from textblob import TextBlob
from wordcloud import WordCloud
# Create a function to get the subjectivity
def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity


# Create two new columns 'Subjectivity' & 'Polarity'
df3['Subjectivity'] = df3['reviews'].apply(getSubjectivity)
df3['Polarity'] = df3['reviews'].apply(getPolarity)

In [ ]:
def getAnalysis(score):
 if score < 0:
  return 'Negative'
 elif score == 0:
  return 'Neutral'
 else:
  return 'Positive'
df3['Analysis'] = df3['Polarity'].apply(getAnalysis)
df3

In [ ]:
df['sentiment']=df3['Analysis']
df['Sentiment']=df['Rating'].apply(lambda x:'Negative' if x<3 else np.nan)
df['Sentiment']=df['Sentiment'].fillna(df['sentiment'])
#== 
df['Reviews_Text']=df3['reviews']
#====
df=df.drop(['sentiment','Review_ID'],axis=1)

In [ ]:
df_copy=df.copy()
df.head()

<a id="3"></a>

<h1 style="color:crimson;background-color:white;font-family:Pacifico;font-size:400%;text-align:center">Data Anlaysis & Visualizations</h1>

<h1 style="color:crimson;background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:center">ـــــــــــ</h1>

* <h1 style="background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Reviews by years</h1>

In [ ]:
sns.set_style("whitegrid")
#== 
df3=df.groupby('year',as_index=False).agg({'review length':'sum'})
plt.figure(figsize=(14,7))
plt.plot(df3['year'] ,df3['review length'],marker='o',label='review length by year',color='black');

* There are many reviews in 2015 

* <h1 style="color:steelblue;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Branches per year</h1>

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df,x='year',hue='Branch',palette='Blues');

* <h1 style="color:black;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Review Length average </h1>

In [ ]:
np.median(df['review length'].values)

* <h1 style="color:goldenrod;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Quarter</h1>

In [ ]:
plt.figure(figsize=(10,6));
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharey=False);
sns.countplot(ax=axes[0],data=df,x='quarter',hue='Rating',palette='gnuplot');
axes[0].set_title('Quarter By Ratings');
sns.countplot(ax=axes[1],data=df,x='quarter',palette='gist_heat');
axes[1].set_title('Quarter per reviews');
#==


* there are less reviews in the first quarter of the year 

* <h1 style="color:steelblue;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Country per reviews </h1>

In [ ]:
fig_df=df.groupby('Reviewer_Location',as_index=False).agg({'Branch':'count'}).sort_values('Branch',ascending=False)
fig = px.choropleth(fig_df, 
                    locations='Reviewer_Location', locationmode='country names',
                    color='Branch',
                    color_continuous_scale="portland", hover_data=['Branch'],
                    title='Country - Reviews')
fig.update(layout_coloraxis_showscale=False)
fig.show();


* <h1 style="color:darkblue;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Country - low rates </h1>

In [ ]:
df4 = df.groupby(['Reviewer_Location'],as_index=False).agg({'Rating':'mean'}).sort_values(by='Rating', ascending=True).head(10)
df4.style.background_gradient(cmap="autumn", subset=['Rating'])

* <h1 style="color:green;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Rates-Year</h1>

In [ ]:
one = df.loc[df['Rating']==1]
one=one.groupby('year',as_index=False).agg({'Rating':'sum'})
#== 
two = df.loc[df['Rating']==2]
two=two.groupby('year',as_index=False).agg({'Rating':'sum'})

#== 
three = df.loc[df['Rating']==3]
three=three.groupby('year',as_index=False).agg({'Rating':'sum'})
#=== 
four = df.loc[df['Rating']==4]
four=four.groupby('year',as_index=False).agg({'Rating':'sum'})
#===
fifth = df.loc[df['Rating']==5]
fifth=fifth.groupby('year',as_index=False).agg({'Rating':'sum'})
#============== 
sns.set_style("darkgrid")
plt.figure(figsize=(14,7))
plt.plot(one['year'] ,one['Rating'],marker='o',label='1') 
plt.plot(two['year'] , two['Rating'],color='blue',marker='*',label='2')  
plt.plot(three['year'] ,three['Rating'],color='green',marker='+',label='3') 
plt.plot(four['year'] ,four['Rating'],color='yellow',marker='+',label='4') 
plt.plot(fifth['year'] ,fifth['Rating'],color='gold',marker='+',label='5') 
plt.legend();


* <h1 style="color:tomato;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Rates-Branch</h1>

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(x="Branch", y="Rating", data=df);

* <h1 style="color:indigo;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Sentiments-Branches</h1>

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df,x='Branch',hue='Sentiment',palette='inferno');

* <h1 style="color:firebrick;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Sentiment percentage-Branch</h1>

In [ ]:
df_cal=df.loc[df['Branch']=='Disneyland_California']
df_par=df.loc[df['Branch']=='Disneyland_Paris']
df_hon=df.loc[df['Branch']=='Disneyland_HongKong']

#===== 
label_cal =df_cal['Sentiment'].value_counts(sort = True).index
size_cal= df_cal['Sentiment'].value_counts(sort = True) 
#=====
label_par =df_par['Sentiment'].value_counts(sort = True).index
size_par= df_par['Sentiment'].value_counts(sort = True) 
#==== 
label_hon =df_hon['Sentiment'].value_counts(sort = True).index
size_hon= df_hon['Sentiment'].value_counts(sort = True)
#===




fig, axes = plt.subplots(1, 3, figsize=(20, 7), sharey=False)
fig.suptitle('Branches')
axes[0].pie(size_cal,labels=label_cal,startangle=90,shadow=1,autopct='%1.2f%%',colors=['#228B22','#CC0000','#00BFFF']);
axes[0].set_title('California')
axes[1].pie(size_par,labels=label_par,startangle=90,shadow=1,autopct='%1.2f%%',colors=['#228B22','#CC0000','#00BFFF']);
axes[1].set_title('Paris');
axes[2].pie(size_hon,labels=label_hon,startangle=90,shadow=1,autopct='%1.2f%%',colors=['#228B22','#CC0000','#00BFFF']);
axes[2].set_title('HongKong');

<a id="4"></a>

<h1 style="color:dodgeblue;background-color:white;font-family:Pacifico;font-size:400%;text-align:center">Sentiment in Reviews</h1>

<h1 style="color:dodgeblue;background-color:white;font-family:Pacifico;font-size:400%;text-align:center">All</h1>

<h1 style="color:dodgeblue;background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:center">ـــــــــــ</h1>

In [ ]:
df['Reviews_Text']=df['Reviews_Text'].astype('str')
Reviews_Text = " ".join(txt for txt in df.Reviews_Text)

c_mask = np.array(Image.open("../input/disneywordcloud/micky disney land .jpg"))
#== 
wc = WordCloud(background_color='white', mask=c_mask, mode='RGB',
               width=1000, max_words=1000, height=1000,
               random_state=1, contour_width=1, contour_color='black',colormap='flag')
wc.generate(Reviews_Text)
plt.figure(figsize=(20, 10))
plt.imshow(wc, interpolation='bilinear')
plt.tight_layout(pad=0)
plt.axis('off')
plt.show()

<h1 style="color:green;background-color:white;font-family:Pacifico;font-size:400%;text-align:center">Positive</h1>

<h1 style="color:green;background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:center">ـــــــــــ</h1>

In [ ]:
df_like=df.loc[df['Sentiment']=='Positive']
#===
Reviews_Text = " ".join(insul for insul in df_like.Reviews_Text)

l_mask = np.array(Image.open("../input/like-worldcloud/pngtree-vector-like-icon-png-image_695769.jpg"))
#== 
wc = WordCloud(background_color='white', mask=l_mask, mode='RGB',
               width=1000, max_words=1000, height=1000,
               random_state=1, contour_width=1, contour_color='green',colormap='Spectral')
wc.generate(Reviews_Text)
plt.figure(figsize=(20, 10))
plt.imshow(wc, interpolation='bilinear')
plt.tight_layout(pad=0)
plt.axis('off')
plt.show()

<h1 style="color:red;background-color:white;font-family:Pacifico;font-size:400%;text-align:center">Negative</h1>

<h1 style="color:red;background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:center">ـــــــــــ</h1>

In [ ]:
df_dislike=df.loc[df['Sentiment']=='Negative']
#===
Reviews_Text = " ".join(insul for insul in df_dislike.Reviews_Text)

l_mask = np.array(Image.open("../input/dislike-wordcloud/2.jpg"))
#== 
wc = WordCloud(background_color='white', mask=l_mask, mode='RGB',
               width=1000, max_words=1000, height=1000,
               random_state=1, contour_width=1, contour_color='red',colormap='Reds')
wc.generate(Reviews_Text)
plt.figure(figsize=(20, 10))
plt.imshow(wc, interpolation='bilinear')
plt.tight_layout(pad=0)
plt.axis('off')
plt.show()

<a id="5"></a>
<h1 style="color:orange;background-color:white;font-family:Pacifico;font-size:400%;text-align:center">Words in Negative Reviews</h1>


<h1 style="color:orange;background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:center">ـــــــــــ</h1>

* <h1 style="color:black;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Reviews Words Segmentation</h1>

In [ ]:
df_dislike
review = df_dislike['Reviews_Text'].drop_duplicates()
all_sentences = []

for word in review:
    all_sentences.append(word)

all_sentences

lines = list()
for line in all_sentences:    
    words = line.split()
    for w in words: 
       lines.append(w)
#== 
import re

lines = [re.sub(r'[^A-Za-z0-9]+', '', x) for x in lines]

lines

lines2 = []

for word in lines:
    if word != '':
        lines2.append(word)
#== 
from nltk.stem.snowball import SnowballStemmer
s_stemmer = SnowballStemmer(language='english')

stem = []
for word in lines2:
    stem.append(s_stemmer.stem(word))
#== 
import spacy
nlp = spacy.load('en_core_web_lg')
stem2 = []

for word in stem:
    if word not in nlp.Defaults.stop_words:
        stem2.append(word)
#== 
df = pd.DataFrame(stem2)
df = df[0].value_counts()
#== 
df = df[:20,]
#== 
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
#====== 
def show_ents(doc):
    if doc.ents:
        for ent in doc.ents:
            print(ent.text + ' - ' + ent.label_ + ' - ' + str(spacy.explain(ent.label_)))
#======
nlp = spacy.load('en_core_web_sm') 
nlp.max_length = 2000000000000
#=====
str1 = " " 
stem2 = str1.join(lines2)

stem2 = nlp(stem2)

label = [(X.text, X.label_) for X in stem2.ents]

df6 = pd.DataFrame(label, columns = ['Word','Entity'])

In [ ]:
df6['Entity'].value_counts()
negative= df6.groupby(['Entity']).size().to_frame('count').reset_index()
negative.sort_values(by='count', ascending=False)[:10]
fig = px.pie(negative, values='count', names='Entity')
fig.show()

* <h1 style="color:black;background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:left"> Time and Dates words constitute 40 % of words in negative reviews texts 
</h1>

* <h1 style="color:dodgerblue;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Time Words ⏱️</h1>

In [ ]:
TIME = df6.query("Entity == 'TIME'")
TIME =TIME['Word'].value_counts()
TIME = TIME[:10]
sns.set_style("darkgrid")
plt.figure(figsize=(14,5));
TIME = sns.barplot(x=TIME.index,y= TIME.values, alpha=0.8,palette="coolwarm");
plt.title('Time Words in negative reviews',fontsize=15);
plt.show();

* <h1 style="color:black;background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:left">Making visitors waits about one hour or more drives them to write negative reviews</h1>

* <h1 style="color:green;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Name Words 🧝</h1>

In [ ]:
PERSON = df6.query("Entity == 'PERSON'")
PERSON =PERSON['Word'].value_counts()
PERSON = PERSON[:10]
sns.set_style("darkgrid")
plt.figure(figsize=(14,5));
PERSON = sns.barplot(x=PERSON.index,y= PERSON.values, alpha=0.8,palette="summer");
plt.title('People name Words in negative reviews',fontsize=15);
plt.show();

### Cehck peter pan negative reviews 

In [ ]:
df_copy['check']=df_copy['Review_Text'].apply(lambda word:word.count('peter pan'))
df_don=df_copy.loc[df_copy['check']>0]
print(df_don['Review_Text'].values[3:4])
print('\n')
print(df_don['Review_Text'].values[4:5])
print('\n')
print(df_don['Review_Text'].values[47:48])


* <h1 style="color:black;background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:left">Visitors of Disneyland California complain about the length of the queues for Peter Pan's game</h1>
* <h1 style="color:black;background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:left">Visitors of Disneyland Paris Complain about the shut of peter pan game </h1>


![1.jpg](https://i.postimg.cc/WbFHf69F/1.jpg)

* <h1 style="color:dodgerblue;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Ordinal Words 🔢</h1>

In [ ]:
ORDINAL = df6.query("Entity == 'ORDINAL'")        
ORDINAL =ORDINAL['Word'].value_counts()
ORDINAL = ORDINAL[:10]
sns.set_style("white")
plt.figure(figsize=(14,5));
PERSON = sns.barplot(x=ORDINAL.index,y= ORDINAL.values, alpha=0.8,palette="terrain");
plt.title('Ordinal Words in negative reviews',fontsize=15);
plt.show();

* <h1 style="color:crimson;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Organizations Words 🏫</h1>


In [ ]:
ORG = df6.query("Entity == 'ORG'")        
ORG =ORG['Word'].value_counts()
ORG = ORG[:10]
sns.set_style("darkgrid")
plt.figure(figsize=(14,5));
ORG = sns.barplot(x=ORG.index,y= ORG.values, alpha=0.8,palette="spring");
plt.title('Organizations  Words in negative reviews',fontsize=15);
plt.show();

### Cehck Indiana Jones negative reviews 

In [ ]:
df_copy['check']=df_copy['Review_Text'].apply(lambda word:word.count('indiana jones'))
df_don=df_copy.loc[df_copy['check']>0]
print(df_don['Review_Text'].values[31:32])
print('\n')
print(df_don['Review_Text'].values[7:8])
print('\n')
print(df_don['Review_Text'].values[20:21])

* <h1 style="color:black;background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:left">This game is really special, although it was mentioned in negative reviews but still recommended by visitors</h1>

![indiana-jones-disneyland.jpg](https://i.postimg.cc/q71FFk3Q/indiana-jones-disneyland.jpg)

* <h1 style="color:tan;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Nation Words 🗺️</h1>


In [ ]:
NORP = df6.query("Entity == 'NORP'")        
NORP =NORP['Word'].value_counts()
NORP = NORP[:10]
sns.set_style("dark")
plt.figure(figsize=(12,5));
NORP = sns.barplot(x=NORP.index,y= NORP.values, alpha=1,palette="Reds");
plt.title('Nation Words in negative reviews',fontsize=15);
plt.show();

### Cehck french negative reviews 

In [ ]:
df_copy['check']=df_copy['Review_Text'].apply(lambda word:word.count('french'))
df_don=df_copy.loc[df_copy['check']>0]
df_don=df_don.loc[df_don['Sentiment']=='Negative']
print(df_don['Review_Text'].values[51:52])
print('\n')
print(df_don['Review_Text'].values[5:6])
print('\n')
print(df_don['Review_Text'].values[20:21])

* <h1 style="color:black;background-color:white;font-family:Gill Sans MT;font-size:130%;text-align:left">foreign visitors to Disney Land Paris faces some issues with french language </h1>

* <h1 style="color:black;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Location Words 📍</h1>


In [ ]:
LOC = df6.query("Entity == 'LOC'")        
LOC =LOC['Word'].value_counts()
LOC = LOC[:10]
sns.set_style("darkgrid")
plt.figure(figsize=(14,5));
NORP = sns.barplot(x=LOC.index,y= LOC.values, alpha=1,palette="flag");
plt.title('location Words in negative reviews',fontsize=15);
plt.show();

### Cehck pirates of the caribbean negative reviews 

In [ ]:
df_copy['check']=df_copy['Review_Text'].apply(lambda word:word.count('caribbean'))
df_don=df_copy.loc[df_copy['check']>0]
df_don=df_don.loc[df_don['Sentiment']=='Negative']
print(df_don['Review_Text'].values[1:2])
print('\n')
print(df_don['Review_Text'].values[5:6])
print('\n')
print(df_don['Review_Text'].values[6:7])

* Seems that Pirates of the Caribbean had a lot of breakdowns 

![2.jpg](https://i.postimg.cc/KYg14YDs/2.jpg)

* <h1 style="color:green;background-color:whitesmoke;font-family:Gill Sans MT;font-size:200%;text-align:left">Money Words 💰</h1>

In [ ]:

PRODUCT = df6.query("Entity == 'MONEY'")        
PRODUCT =PRODUCT['Word'].value_counts()
PRODUCT = PRODUCT[:10]
sns.set_style("white")
plt.figure(figsize=(24,5));
NORP = sns.barplot(x=PRODUCT.index,y= PRODUCT.values, alpha=1,palette="ocean");
plt.title('Money Words in negative reviews',fontsize=15);
plt.show();

<h1 style="color:dodgeblue;background-color:white;font-family:Pacifico;font-size:400%;text-align:center">Thanks</h1>


<img src="https://media.giphy.com/media/wlBC82Ryvux68/giphy.gif">